In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('/kaggle/input/facebook-recruiting-iv-human-or-bot/train.csv.zip')
test = pd.read_csv('/kaggle/input/facebook-recruiting-iv-human-or-bot/test.csv.zip')
# bids = pd.read_csv('/kaggle/input/facebook-recruiting-iv-human-or-bot/bids.csv.zip')

In [ ]:
bids.head() # merge - 두 개 데이터 프레임 병합, 하나의 id에 대푯값(평균 등)을 넣어줌

In [ ]:
# 20:03
# 힌트 -  인간은 일주일 동안 온라인에서 구매했을 시, 100번이상은 안넘어감 

In [ ]:
bids['bidder_id'].value_counts() # 각각 id별로 구매를 몇번했는지 count를 추가해주면 모델의 점수가 높아질 것임

In [ ]:
bids.groupby('bidder_id')['bid_id'].count().reset_index() # 쉽게 df 만드는법
# 행 6614 인데 몇개의 계정은 구매를 하지 않음 

In [ ]:
bid_count = bids.groupby('bidder_id')['bid_id'].count().reset_index()

In [ ]:
#ip는 사람도 바뀌는 경우도 잇음
#url은 점수에 도움이 안되는 데 그 이유는 다른 데이터셋에서 url의 정보를 담고 분석에 들어가서
#time - 구매와 구매 사이의 구간이 짧으면 로봇일 확률
# time  - 유행하는 시즌/집중 시기이 있을 것임 -> 연월일로 바꿔줘서 이를 파악  
#merchandise - 이건 아이디당 하나라 count가 적용이 안됨. 이에, 그냥 아이디랑 올려주는데 나중에 레이블인코더 실시

In [ ]:
# 3/5 변수 추가
#time - 구매와 구매 사이의 구간이 짧으면 로봇일 확률
# 하나의 id당 최초구매 최근(마지막) 구매
# 이를 id당 구매횟수로 나눠줌 

time_gap = bids.groupby('bidder_id')['time'].apply(lambda x : x.max() - x.min())


# country nunique / device nunique들이 ip에 이미 포함됨. 이에 ip를 추가하면 과대적합 

In [ ]:
from datetime import datetime

# bids['time'].apply(lambda x : datetime.fromtimestamp(x/7000000))
# year 311228 is out of range 천에서 십만으로 나눠줌
# 5000년도를 그대로 해도 되지만
# 7백만

bids['datetime'] = bids['time'].apply(lambda x : datetime.fromtimestamp(x/7000000))
bids.head()

In [ ]:
bids['year'] = bids['datetime'].dt.year
bids['month'] = bids['datetime'].dt.month
bids['day'] = bids['datetime'].dt.day
bids.head()

# 하나의 id당 여러 타임들이 가져올 수 있어서 연도의평균 월의 평균이 필요



In [ ]:
date_mean = bids.groupby('bidder_id')['year','month','day'].mean().reset_index()

In [ ]:
device_nunique = bids.groupby('bidder_id')['device'].nunique().reset_index()
country_nunique = bids.groupby('bidder_id')['country'].nunique().reset_index()
auction_nunique = bids.groupby('bidder_id')['auction'].nunique().reset_index()


In [ ]:
nunique = bids.groupby('bidder_id')['auction','device','country'] # 여기서 미리 만들고 merge


In [ ]:
# alldata = pd.concat([train, test])
# alldata = pd.merge(alldata,bid_count,on='bidder_id',how='left')
# alldata.h

In [ ]:
alldata = pd.concat([train, test])
alldata = pd.merge(alldata,bid_count,on='bidder_id',how='left')
alldata = pd.merge(alldata,auction_nunique,on='bidder_id',how='left')
alldata = pd.merge(alldata,device_nunique,on='bidder_id',how='left')
alldata = pd.merge(alldata,country_nunique,on='bidder_id',how='left')
alldata = pd.merge(alldata,time_gap.reset_index(),on='bidder_id',how='left')
alldata = pd.merge(alldata,date_mean,on='bidder_id',how='left') # 활동하는 패턴/시기 파악 
alldata.head()



In [ ]:
alldata['time'] = alldata['time']/alldata['bid_id']  #교호작용
alldata.head()

In [ ]:
alldata3 = alldata[alldata['bid_id']<20000]

In [ ]:
# 그래프를 통해 bid_id가 유의미한 데이터인지 확인
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20,12)) # 문자형(0, 1) / 숫자형(bid id) -> 
# sns.boxplot(alldata['outcome'],alldata['bid_id']) # 아웃라이어 떄문에 그래프가 이상함 -> 조건을 줘서 얘네 제외
sns.boxplot(alldata3['outcome'],alldata3['bid_id']) # 대부분의 로봇은 구매횟수가 높구나

In [ ]:
# 그래프를 통해 bid_id가 유의미한 데이터인지 확인
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20,12)) # 문자형(0, 1) / 숫자형(bid id) -> 
# sns.boxplot(alldata['outcome'],alldata['bid_id']) # 아웃라이어 떄문에 그래프가 이상함 -> 조건을 줘서 얘네 제외
sns.boxplot(alldata['outcome'],alldata['auction']) # 대부분의 로봇은 구매횟수가 높구나

In [ ]:
alldata4 = alldata[alldata['time']<5000000000]

In [ ]:
# 그래프를 통해 bid_id가 유의미한 데이터인지 확인
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20,12)) # 문자형(0, 1) / 숫자형(bid id) -> 
# sns.boxplot(alldata['outcome'],alldata['bid_id']) # 아웃라이어 떄문에 그래프가 이상함 -> 조건을 줘서 얘네 제외
sns.boxplot(alldata['outcome'],alldata4['time']) # 인간과 로봇간의 차이가 확연한지 

In [ ]:
alldata3 = alldata[alldata['auction']<20000]

In [ ]:
alldata['payment_account'] = alldata['payment_account'].apply(lambda x:x[:5]) # 

In [ ]:
alldata['address'].sort_values(ascending = False).head(50)

In [ ]:
alldata['address'] = alldata['address'].apply(lambda x:x[:7])

In [ ]:
alldata.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
alldata['payment_account'] = le.fit_transform(alldata['payment_account'])
alldata['address'] = le.fit_transform(alldata['address'])

In [ ]:
alldata.head()

In [ ]:
#import matplotlib.pyplot as plt
#import seaborn as sns

#plt.figure(figsize = (20,12))
# sns.boxplot(train3['week'],train['Weekly_Sales'])

#sns.distplot(np.log(train3['Weekly_Sales']))

#plt.train['year']
#plt.show()

In [ ]:
alldata2 = alldata.drop(['outcome', 'bidder_id'], axis = 1)

In [ ]:
alldata2 = alldata2.fillna(-1) # 왠만하면 다른 데이터랑 안겹치게 넣어줌 -> -1 

In [ ]:
alldata2.head()

In [ ]:
alldata2.shape

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
alldata3

In [ ]:
alldata2

In [ ]:
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(n_jobs = 4)
# rf.fit(train2, train['outcome'])
# # result = rf.predict(test2)
# result = rf.predict_proba(test2)

# 3/5
# 데이터 복잡도 파악 / 데이터를 모델에 최적합하게 적용 > 데이터의 특징 도출 
# 데이터 복잡도 파악 방법
# 1) 데이터의 갯수 (만개에서 몇만개면 어느정도 복잡 이때 depth =6이 적당, 여기선 2000개라 단순한편 = 행 갯수)
# 2) 데이터의 칼럼 갯수 (10~30개면 중간정도 복잡도 30~ 100개면 복잡, 여기선 칼럼이 단순한편)
# 3) 칼럼 중 카테고리형 파악하고 이 범주(종류의 갯수)가 몇개인지 = 트리 모델이 많은 질문 필요 = 복잡 ***
# ** 4) y값 학습 시, 카테고리형 칼럼이 얼마나 중요하게 학습했는지 / 해당 칼럼이 중요치 않으면 아예 학습을 안하게됨 => 복잡하다고 느끼지 않음 -> 추가로 처리 필요
# 항상 데이터셋이 복잡한지 안복잡한지 파악
# 이 데이터셋은 단순한편 max_depth를 높게 해주면 바로 과대적합 문제가 일어남 -> 이에 max_depth를 일부러 낮게 맞춰줌, 7정도까지만 


from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = 4, max_depth=7, n_estimators=170) 
# max_depth 기본값이 -1, none 
# 하나의 나무에서 알 수 있는 정보가 줄어듬 예전에는 무한정이었음
# 이에 depth (한 가지) 자체가 줄었기때문에 한 가지의 나무의 갯수(n_estimators)를 늘려줌 
rf.fit(train2, train['outcome'])
# result = rf.predict(test2)
result = rf.predict_proba(test2)

# 데이터셋이 단순해 이에 맞춰 모델링 진행하니 결과가 좋아짐


In [ ]:
result #쉼표를 기준으로 왼쪽이 인간이 확률 = 0 / 오른쪽이 로봇일 확률 = 1 -> 대부분의 대회는 오른쪽에 있는 컬럼을 제출 
result = result[:,1]
# 3번째가 0.38로 높음, 로봇일 확률이 큰거임 0.05 가 원래 

In [ ]:
train['outcome'].value_counts()

In [ ]:
sub = pd.read_csv('/kaggle/input/facebook-recruiting-iv-human-or-bot/sampleSubmission.csv')
sub['prediction'] = result
sub.head()

In [ ]:
sub.to_csv('class4.csv', index = False)